In [ ]:
import yaml
from omegaconf import OmegaConf
from yolo_ev.module.model_module import ModelModule
from yolo_ev.module.data_module import DataModule

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import loggers as pl_loggers


save_dir = './result'

yaml_file = "./../config/param.yaml"
with open(yaml_file, 'r') as file:
    config = yaml.safe_load(file)
config = OmegaConf.create(config)

data = DataModule(config)
model = ModelModule(config)



In [ ]:
data.setup('fit')

In [ ]:
from torch.utils.data import DataLoader
dataset = data.train_dataset
dataloader = DataLoader(dataset)


In [ ]:
data_iter = iter(dataloader)
img, target, info, id = data_iter._next_data()
print(img.shape)
print(target.shape)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_img_and_bboxes(img, target):
    """
    画像とターゲット (バウンディングボックス) を可視化する関数
    
    Parameters:
    - img: テンソル (C, H, W) 形式の画像データ
    - target: (N, 5) 形式のターゲットデータ。各行が [cls_id, cx, cy, w, h] 形式で表される。
    """
    
    # imgをnumpyに変換して順番を変える
    img_np = img.permute(1, 2, 0).cpu().numpy()  # [C, H, W] -> [H, W, C]に変換
    if img_np.max() > 1.0:
        img_np = img_np / 255.0

    # 画像サイズを取得
    img_height, img_width, _ = img_np.shape

    # imgの表示
    plt.figure(figsize=(10, 10))
    plt.imshow(img_np)
    plt.axis('off')

    # targetに含まれるbounding boxをプロット
    for i in range(target.shape[0]):  # targetの50はboxの数
        cls_id, cx, cy, w, h = target[i]
        # バウンディングボックスの座標を計算 (cx, cy) は中心座標、w, h は幅と高さ

        # 正確な座標計算を確認するため、スケーリングに注意
        x1 = (cx - w / 2)  # 左上のx座標
        y1 = (cy - h / 2)  # 左上のy座標
        x2 = (cx + w / 2)  # 右下のx座標
        y2 = (cy + h / 2) 

        # バウンディングボックスの描画
        plt.gca().add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, 
                                          fill=False, edgecolor='red', linewidth=2))

        # クラスIDをバウンディングボックスの上に表示（任意）
        plt.text(x1, y1, f'Class: {int(cls_id)}', color='yellow', fontsize=12, 
                 bbox=dict(facecolor='red', alpha=0.5))

    # プロットの表示
    plt.show()


In [ ]:
visualize_img_and_bboxes(img[0], target[0])

In [ ]:
def vis(img, target):
    """
    画像とターゲット (バウンディングボックス) を可視化する関数
    
    Parameters:
    - img: テンソル (C, H, W) 形式の画像データ
    - target: (N, 5) 形式のターゲットデータ。各行が [cls_id, cx, cy, w, h] 形式で表される。
    """
    
    # imgをnumpyに変換して順番を変える
    img_np = img.transpose(1, 2, 0)
    if img_np.max() > 1.0:
        img_np = img_np / 255.0

    # 画像サイズを取得
    img_height, img_width, _ = img_np.shape

    # imgの表示
    plt.figure(figsize=(10, 10))
    plt.imshow(img_np)
    plt.axis('off')

    # targetに含まれるbounding boxをプロット
    for i in range(target.shape[0]):  # targetの50はboxの数
        cls_id, cx, cy, w, h = target[i]
        # バウンディングボックスの座標を計算 (cx, cy) は中心座標、w, h は幅と高さ

        # 正確な座標計算を確認するため、スケーリングに注意
        x1 = (cx - w / 2)  # 左上のx座標
        y1 = (cy - h / 2)  # 左上のy座標
        x2 = (cx + w / 2)  # 右下のx座標
        y2 = (cy + h / 2) 

        # バウンディングボックスの描画
        plt.gca().add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, 
                                          fill=False, edgecolor='red', linewidth=2))

        # クラスIDをバウンディングボックスの上に表示（任意）
        plt.text(x1, y1, f'Class: {int(cls_id)}', color='yellow', fontsize=12, 
                 bbox=dict(facecolor='red', alpha=0.5))

    # プロットの表示
    plt.show()

In [ ]:
img, target, info, id = dataset[0]

In [ ]:
img.shape

In [ ]:
vis(img, target)